# Movie Studio Analysis: Understanding Box Office Success
## Business Understanding
The company is planning to launch a new movie studio, but lacks experience in movie production. The goal of this analysis is to explore current trends in the film industry and provide actionable insights that can guide the studio's strategy. This analysis focuses on identifying factors that contribute to box office success, including genre, budget, release timing, and the impact of key personnel such as directors, actors, and writers.
    

## Data Understanding
The analysis is based on multiple datasets related to movie budgets, revenues, genres, release dates, and key personnel (directors, actors, and writers).

These datasets include:
   - **Box Office Mojo (BOM) Movie Gross**: Contains information on domestic and foreign box office revenues.
   - **TMDB Movies**: Includes information on movie popularity and ratings.
   - **IMDb Database**: Provides detailed information on directors, actors, writers, and other key personnel involved in the movies.
    

## Data Preparation
In this section, we will load, clean, and merge the datasets to prepare them for analysis.

In [2]:
# Library imports
import pandas as pd
import sqlite3
from zipfile import ZipFile
import numpy as np
import scipy.stats as stats

In [3]:
# Load the datasets
tmdb_movies = pd.read_csv("zippedData/tmdb.movies.csv.gz")
tn_movies = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

In [4]:
# Loading IMDb SQLite database

# Unzip the sqlite db file if not already done
with ZipFile("zippedData/im.db.zip", 'r') as zObject:
    zObject.extractall("zippedData/")

# Creating the connection
conn = sqlite3.connect("zippedData/im.db")

# Loading data for directors, actors, and writers filtering for US movies in English

# Queries
query_directors = """
SELECT mb.*, mr.averagerating, mr.numvotes, p.primary_name, p.birth_year, p.death_year, p.primary_profession
FROM movie_basics AS mb
JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
JOIN principals AS pr ON mb.movie_id = pr.movie_id
JOIN persons AS p ON pr.person_id = p.person_id
JOIN movie_akas AS ma ON mb.movie_id = ma.movie_id
WHERE ma.region = 'US'
AND pr.category = 'director'
AND ma.language = 'en';
"""

query_actors = """
SELECT mb.*, mr.averagerating, mr.numvotes, p.primary_name, p.birth_year, p.death_year, p.primary_profession
FROM movie_basics AS mb
JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
JOIN principals AS pr ON mb.movie_id = pr.movie_id
JOIN persons AS p ON pr.person_id = p.person_id
JOIN movie_akas AS ma ON mb.movie_id = ma.movie_id
WHERE ma.region = 'US'
AND pr.category = 'actor'
AND ma.language = 'en';
"""

query_writers = """
SELECT mb.*, mr.averagerating, mr.numvotes, p.primary_name, p.birth_year, p.death_year, p.primary_profession
FROM movie_basics AS mb
JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
JOIN principals AS pr ON mb.movie_id = pr.movie_id
JOIN persons AS p ON pr.person_id = p.person_id
JOIN movie_akas AS ma ON mb.movie_id = ma.movie_id
WHERE ma.region = 'US'
AND pr.category = 'writer'
AND ma.language = 'en';
"""

query_basics = """
SELECT *
FROM movie_basics
"""

query_ratings = """
SELECT *
FROM movie_ratings
"""

# Execute queries and assign to dataframes
directors_merged = pd.read_sql_query(query_directors, conn)
actors_merged = pd.read_sql_query(query_actors, conn)
writers_merged = pd.read_sql_query(query_writers, conn)
movie_basics = pd.read_sql_query(query_basics, conn)
movie_ratings = pd.read_sql_query(query_ratings, conn)

# Close the connection
conn.close()

## Data Cleaning
Here I'll go through the process of cleaning the data by handling missing values, removing duplicate information, recasting data types, and feature engineering

In [5]:
# Cleaning IMDb data
def clean_imdb_data(df):
    # Handle missing values (dropping rows with missing ratings or votes)
    df = df.dropna(subset=['averagerating', 'numvotes', 'primary_name'])
    
    # Convert numvotes to integers
    df['numvotes'] = df['numvotes'].astype(int)
    
    # Filter out movies with less than 1000 votes
    df = df[df['numvotes'] >= 1000]
    
    return df

directors_cleaned = clean_imdb_data(directors_merged)
actors_cleaned = clean_imdb_data(actors_merged)
writers_cleaned = clean_imdb_data(writers_merged)

# Extrating the primary genre from the 'genres' column
movie_basics['primary_genre'] = movie_basics['genres'].apply(lambda x: x.split(',')[0] if pd.notnull(x) else None)

In [6]:
# Cleaning TMDB data
def clean_tmdb_data(df):
    # Handle missing values
    df = df.dropna(subset=['popularity', 'vote_count', 'release_date'])
    
    # Convert release_date to datetime
    df['release_date'] = pd.to_datetime(df['release_date'])
    
    # Convert vote_count to integer
    df['vote_count'] = df['vote_count'].astype(int)
    
    # Remove any duplicate values based on movie title and release date
    df = df.drop_duplicates(subset=['title', 'release_date'])
    
    return df

tmdb_cleaned = clean_tmdb_data(tmdb_movies)

In [7]:
# Cleaning 'The Numbers' movie budgets data

# Our selected data already has currency in the proper format but just in case I add more data later this will come in handy
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', ''))
    return x

def clean_tn_movie_budgets(df):
    # Apply the currency cleaning function to the budget and revenue columns
    df['production_budget'] = df['production_budget'].apply(clean_currency)
    df['domestic_gross'] = df['domestic_gross'].apply(clean_currency)
    df['worldwide_gross'] = df['worldwide_gross'].apply(clean_currency)
    
    # Convert release_date to datetime
    df['release_date'] = pd.to_datetime(df['release_date'])
    
    # Handle missing values
    df = df.dropna(subset=['production_budget', 'domestic_gross', 'worldwide_gross'])
    
    return df

tn_movie_budgets_cleaned = clean_tn_movie_budgets(tn_movies)

In [8]:
# Merge TMDB and TN Movie Budgets data
merged_data = pd.merge(tmdb_cleaned, tn_movie_budgets_cleaned, left_on='title', right_on='movie')

# Dropping duplicates based on 'title' and 'release_date'
if 'release_date' in merged_data.columns:
    merged_data = merged_data.drop_duplicates(subset=['title', 'release_date'])
    print("Dropped duplicates based on 'title' and 'release_date'.")
else:
    print("Error: 'release_date' column is missing in the merged data.")
    
# Merging IMDb data
# Ensuring correct data types
movie_basics['primary_title'] = movie_basics['primary_title'].astype(str)
merged_data['title'] = merged_data['title'].astype(str)

# Merge the primary genre from IMDb into the merged_data DataFrame
merged_data = pd.merge(merged_data, movie_basics[['primary_title', 'primary_genre']], left_on='title', right_on='primary_title', how='left')

# Drop the extra 'primary_title' column after merging
merged_data = merged_data.drop(columns=['primary_title'])

Error: 'release_date' column is missing in the merged data.


In [9]:
# Adding in ROI columns
merged_data['roi_domestic'] = merged_data['domestic_gross'] / merged_data['production_budget']
merged_data['roi_worldwide'] = merged_data['worldwide_gross'] / merged_data['production_budget']

In [10]:
# Drop duplicates based on 'title' and 'release_date'
merged_data = merged_data.drop_duplicates(subset=['title', 'release_date_x'])

In [11]:
# Final merged dataset preview
merged_data.head()

,Unnamed: 0,genre_ids,id_x,original_language,original_title,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,movie,production_budget,domestic_gross,worldwide_gross,primary_genre,roi_domestic,roi_worldwide
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,2010-03-26,How to Train Your Dragon,165000000.0,217581232.0,494870992.0,Action,1.318674,2.999218
1,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,2010-05-07,Iron Man 2,170000000.0,312433331.0,621156389.0,Action,1.837843,3.653861
2,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,1995-11-22,Toy Story,30000000.0,191796233.0,364545516.0,NaN,6.393208,12.151517
3,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,38,2010-07-16,Inception,160000000.0,292576195.0,835524642.0,Action,1.828601,5.222029
4,5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,17,2010-02-12,Percy Jackson & the Olympians: The Lightning T...,95000000.0,88768303.0,223050874.0,Adventure,0.934403,2.347904


In [12]:
merged_data[merged_data['title'] == 'Home']

,Unnamed: 0,genre_ids,id_x,original_language,original_title,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,movie,production_budget,domestic_gross,worldwide_gross,primary_genre,roi_domestic,roi_worldwide
818,3525,"[18, 9648]",79580,ru,Dom,1.823,2011-11-11,Home,5.4,11,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
1703,9651,[],237303,en,Home,0.840,2013-11-22,Home,7.0,2,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
2551,14262,"[14, 35, 16, 878, 10751]",228161,en,Home,12.835,2015-03-27,Home,6.8,2372,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
3441,19580,[],409371,en,Home,0.672,2016-08-01,Home,7.0,1,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
3582,20446,"[27, 18, 10751]",381507,en,Home,0.600,2016-03-01,Home,4.7,14,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
3654,20455,[18],388223,en,Home,0.600,2016-03-17,Home,4.5,2,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215
4080,23694,[878],496256,en,Home,0.600,2017-04-30,Home,4.0,1,44,2015-03-27,Home,130000000.0,177397510.0,385997896.0,Drama,1.364596,2.969215


In [13]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2124 entries, 0 to 4420
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Unnamed: 0         2124 non-null   int64         
 1   genre_ids          2124 non-null   object        
 2   id_x               2124 non-null   int64         
 3   original_language  2124 non-null   object        
 4   original_title     2124 non-null   object        
 5   popularity         2124 non-null   float64       
 6   release_date_x     2124 non-null   datetime64[ns]
 7   title              2124 non-null   object        
 8   vote_average       2124 non-null   float64       
 9   vote_count         2124 non-null   int64         
 10  id_y               2124 non-null   int64         
 11  release_date_y     2124 non-null   datetime64[ns]
 12  movie              2124 non-null   object        
 13  production_budget  2124 non-null   float64       
 14  domestic_gros

## Hypothesis testing

In [16]:
# Does genre significantly impact ROI?

# Drop missing values for the relevant columns
genre_roi_data = merged_data.dropna(subset=['primary_genre', 'roi_domestic', 'roi_worldwide'])

# Group by primary genre and calculate the mean revenue for each genre
genre_roi = genre_roi_data.groupby('primary_genre')['roi_worldwide'].mean()

# Perform ANOVA to test if the means are significantly different
anova_results = stats.f_oneway(*[genre_roi_data[genre_roi_data['primary_genre'] == genre]['roi_worldwide']
                                 for genre in genre_roi.index])

# Output the ANOVA results
print("ANOVA Results for Genre Impact on Worldwide ROI:")
print(f"F-statistic: {anova_results.statistic}, p-value: {anova_results.pvalue}")

# Interpretation
if anova_results.pvalue < 0.05:
    print("Reject the null hypothesis: There is a significant difference in worldwide ROI across genres.")
else:
    print("Fail to reject the null hypothesis: No significant difference in worldwide ROI across genres.")

ANOVA Results for Genre Impact on Worldwide ROI:
F-statistic: 4.885498161043862, p-value: 1.9515694973538687e-10
Reject the null hypothesis: There is a significant difference in worldwide ROI across genres.
